## TensorFlow For Beginners - Part 6

We are almost at the end of our initial discussion. We are now going to take a closer look at tensorflow variables.

# Variables

Anyone with programming background is familiar with the term variable. So they are again tensors but variable. Its that simple. TensorFlow documentation defines variable as the best way to represent shared, persistent state manipulated by the program.

So, lets look at how we can use tensorflow variables in our programs.

### Creating variables the old way

First take a look at how variables were initially created using tensorflow:

In [1]:
import tensorflow as tf

s = tf.Variable(2, name="scalar") 
m = tf.Variable([[0, 1], [2, 3]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))

<b>Note:</b> It's written tf.constant with lowercase ‘c’ but tf.Variable with uppercase ‘V’. It’s because tf.constant is an op, while tf.Variable is a class with multiple ops.

### Creating variables the right way

The previous method is discouraged and TensorFlow recommends that we use tf.get_variable, which allows for easy variable sharing. 

<b>Syntax:</b> <br />
tf.get_variable( <br />
    &nbsp;&nbsp;&nbsp;&nbsp;name, <br />
    &nbsp;&nbsp;&nbsp;&nbsp;shape=None, <br />
    &nbsp;&nbsp;&nbsp;&nbsp;dtype=None, <br />
    &nbsp;&nbsp;&nbsp;&nbsp;initializer=None, <br /> )

We have omitted the attributes, mentioning only frequently used ones.

<b>Note:</b> When we use tf.constant as an initializer, we don’t need to provide shape.

In [2]:
# If variables with same name are already present then we get an error. 
# So we reset the default graph.
tf.reset_default_graph()

# Defining variables the right way
s = tf.get_variable("scalar", initializer=tf.constant(2)) 
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

### Initialization

Variable initializers must be run explicitly before other ops in your model can be run. The easiest way to do that is to add an op that runs all the variable initializers, and run that op before using the model.

We generally use <b>tf.global_variables_initializer()</b> to add an op to run variable initializers. Only run that op after you have fully constructed your model and launched it in a session.

We are going to create a method which will do this whenever we try to print the value of a tensorflow variable.

In [3]:
def run(variable):    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        print(sess.run(variable))

In [4]:
# Let us print the value of the variable 's' from above
run(s)

2


We can also run the initializer in other ways as well. Some methods are shown below.

In [5]:
# Initializing only the required variables

with tf.Session() as sess:
    init_op = tf.variables_initializer([m, W])
    sess.run(init_op)
    print(m.eval())

[[0 1]
 [2 3]]


In [6]:
# We can even initialize only the required variable

x = tf.get_variable("x", initializer=tf.constant(5))

with tf.Session() as sess:
    init_op = x.initializer
    sess.run(init_op)
    print(x.eval())

5


### Initializing from other variables

We sometimes need to initialize a variable from the initial value of another variable. As the operation added by tf.global_variables_initializer() initializes all variables in parallel we have to be careful when this is needed.

To initialize a new variable from the value of another variable use the other variable's initialized_value() property. We can use the initialized value directly as the initial value for the new variable, or you can use it as any other tensor to compute a value for the new variable.

In [7]:
# Create a new variable

tf.reset_default_graph()

p = tf.get_variable("p", initializer=tf.constant(10))

q = tf.get_variable("q", initializer=p.initialized_value() * 2)

run(q)

20


### Assign value

As the name suggests it is used to assign values to variables after initialization.

In [8]:
tf.reset_default_graph()

W = tf.get_variable('W', initializer=tf.constant(10))

print("W = ", end="")
run(W)

# Assignment operation
assign_op = W.assign(100)

print("Value of W after assignment = ", end="")
with tf.Session() as sess:
    sess.run(assign_op)
    print(W.eval())

W = 10
Value of W after assignment = 100


Lets look at two other assign functions:

1) <b>tf.assign_add:</b> Adding mentioned value to the the variable and assign the sum to the variable. <br />
    &nbsp;&nbsp;&nbsp;&nbsp;<code>variable = variable + value</code>
    
2) <b>tf.assign_sub:</b> Subtract mentioned value from the the variable and assign the difference to the variable. <br />
    &nbsp;&nbsp;&nbsp;&nbsp;<code>variable = variable - value</code>

In [9]:
tf.reset_default_graph()

a = tf.get_variable("a", initializer=tf.constant(10))

print("Value of variable: ", end="")
run(a)

total = a.assign_add(5)

print("After adding 5 to the variable, value of variable = ", end="")
run(total)

diff = a.assign(10)

print("After substracting 10 from the variable, value of variable = ", end="")
run(diff)

Value of variable: 10
After adding 5 to the variable, value of variable = 15
After substracting 10 from the variable, value of variable = 10


Because TensorFlow sessions maintain values separately, each Session can have its own current value for a variable defined in a graph.

In [10]:
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()
sess1.run(W.initializer)
sess2.run(W.initializer)
print(sess1.run(W.assign_add(10)))  # >> 20
print(sess2.run(W.assign_sub(2)))   # >> 8
print(sess1.run(W.assign_add(100))) # >> 120
print(sess2.run(W.assign_sub(50)))  # >> -42
sess1.close()
sess2.close()

20
8
120
-42


This is all about tensorflow variables. It is important to get acquainted with variables as they are widely used in tensorflow programs. 

## Bonus Section

### 1. Interactive sessions

As you may have already felt that it is tedious to always invoke a session to evaluate a variable and it also increases the code complexity. In such situations we can use Interactive Sessions.

In [11]:
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
# We can just use 'c.eval()' without passing 'sess'
print(c.eval())
sess.close()

30.0


### 2. Saving and Restoring variables

Often it is required to save a variable and use them in some other program. Lets see how to do this. 

#### Saving Variables

Create a Saver with tf.train.Saver() to manage all variables in the model.

In [12]:
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", initializer=tf.constant(5))
v2 = tf.get_variable("v2", initializer=tf.constant(10))

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  
  save_path = saver.save(sess, "model.ckpt")
  print("Model saved in file: %s" % save_path)

Model saved in file: model.ckpt


#### Restoring Variables

The same Saver object is used to restore variables. Note that when you restore variables from a file you do not have to initialize them beforehand.

In [13]:
# Deleting all the nodes from the graph
tf.reset_default_graph()

with tf.Session() as sess:
  # Restore variables from disk.
  saver = tf.train.import_meta_graph('model.ckpt.meta')
  saver.restore(sess, tf.train.latest_checkpoint('./'))
  print("Model restored.")

INFO:tensorflow:Restoring parameters from ./model.ckpt
Model restored.


In [14]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

print_tensors_in_checkpoint_file(file_name='./model.ckpt', tensor_name = '', all_tensors = True)

tensor_name:  v1
5
tensor_name:  v2
10


In [15]:
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("v1:0")
w2 = graph.get_tensor_by_name("v2:0")

In [16]:
print("w1 = v1 = ", end='')
run(w1)

print("w2 = v2 = ", end='')
run(w2)

w1 = v1 = 5
w2 = v2 = 10
